# Exploration 11 - 뉴스기사 요약하기 

이번 프로젝트는 자연어 프로젝트 중, seq2seq 모델의 어텐션(Attention) 메커니즘을 사용하여 문장을 요약하는 프로젝트이다.   
seq2seq 모델은 LSTM을 기반으로 한 모델로 인코더 부분과 디코더 부분으로 나누어 입력(인풋)을 인코더 부분에 넣고 LSTM이 학습하는 과정을 거친 후 이를 컨텍스트 벡터(context vector)형태로 변환한다. 그 다음으로 변환된 context vector을 디코더 부분에 넣어 예측할 문장을 생성하는 것이다.   
   
기본적인 개념에 대한 설명이었고, 실제로 위 모델을 사용하여 본문에서 중요한 단어를 LSTM 어텐션 메커니즘을 사용하여 학습한 후 얼만큼의 요약문을 만들어낼 수 있는지 확인해보겠다.

## 필요한 모듈 import

In [5]:
import nltk
nltk.download('stopwords')

import numpy as np
import pandas as pd
import os
import re
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
from bs4 import BeautifulSoup 
from tensorflow.keras.preprocessing.text import Tokenizer 
from tensorflow.keras.preprocessing.sequence import pad_sequences
import urllib.request

[nltk_data] Downloading package stopwords to /home/aiffel-
[nltk_data]     dj58/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


# Step 1. 데이터 준비하기

In [3]:
import urllib.request
urllib.request.urlretrieve("https://raw.githubusercontent.com/sunnysai12345/News_Summary/master/news_summary_more.csv", filename="news_summary_more.csv")
data = pd.read_csv('news_summary_more.csv', encoding='iso-8859-1')

뉴스요약에 사용될 데이터를 준비한다.   
데이터는 위 코드를 통해 다운받을 수 있으며, data라는 변수에 pandas를 사용하여 불러온 후 Step 2에서 전처리를 진행하겠다.

In [6]:
print(len(data))

98401


전체 데이터는 98401개를 가지고 있다.   
데이터 형태를 살펴보면, headlines와 text부분으로 나뉘는데, text는 뉴스의 본문 내용이며 headlines는 뉴스의 제목이다.   
headlines는 앞으로 학습할 모델에서 lable로 지정되는데 정확한 요약문은 아니더라도 본문에 내용을 충분히 반영하고 있다고 생각한다.

In [7]:
data.head(10)

,headlines,text
0,upGrad learner switches to career in ML & Al w...,"Saurav Kant, an alumnus of upGrad and IIIT-B's..."
1,Delhi techie wins free food from Swiggy for on...,Kunal Shah's credit card bill payment platform...
2,New Zealand end Rohit Sharma-led India's 12-ma...,New Zealand defeated India by 8 wickets in the...
3,Aegon life iTerm insurance plan helps customer...,"With Aegon Life iTerm Insurance plan, customer..."
4,"Have known Hirani for yrs, what if MeToo claim...",Speaking about the sexual harassment allegatio...
5,Rahat Fateh Ali Khan denies getting notice for...,Pakistani singer Rahat Fateh Ali Khan has deni...
6,"India get all out for 92, their lowest ODI tot...",India recorded their lowest ODI total in New Z...
7,Govt directs Alok Verma to join work 1 day bef...,Weeks after ex-CBI Director Alok Verma told th...
8,Called PM Modi 'sir' 10 times to satisfy his e...,Andhra Pradesh CM N Chandrababu Naidu has said...
9,"Cong wins Ramgarh bypoll in Rajasthan, takes t...",Congress candidate Shafia Zubair won the Ramga...


# Step 2. 데이터 전처리하기

## 중복 샘플과 NULL 값이 존재하는 샘플 제거

In [9]:
print('Text 열에서 중복을 배제한 유일한 샘플의 수 :', data['text'].nunique())
print('Summary 열에서 중복을 배제한 유일한 샘플의 수 :', data['headlines'].nunique())

Text 열에서 중복을 배제한 유일한 샘플의 수 : 98360
Summary 열에서 중복을 배제한 유일한 샘플의 수 : 98280


`nunique`를 사용하여 중복되지 않은 값을 확인할 수 있다.

In [10]:
data.drop_duplicates(subset = ['text'], inplace = True)
print('전체 샘플수 :',(len(data)))

전체 샘플수 : 98360


중복된 값들을 `drop_duplicates` 메서드를 사용하여 날려준다.   
하지만 NULL로 중복되어있는 한개의 값이 남아있으므로 확인 후 drop 해주도록 한다.

In [11]:
print(data.isnull().sum())

headlines    0
text         0
dtype: int64


NULL값은 없는걸로 확인된다.

## 텍스트 정규화와 불용어 제거